In [13]:
from llama_index.core.tools import FunctionTool
from llama_index.llms.groq import Groq
import json

llm = Groq(model="llama3-70b-8192", api_key="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")


In [2]:
from typing import Dict, Optional
from datetime import datetime

def complaint_classifier(complaint: dict) -> tuple[str,str]:
    """
        You are an intelligent complaint classification system working for Rail Madad, the IRCTC's complaint grievance platform. Your task is to:

    Determine the priority of a complaint (Emergency, High, Medium, Low).

    Emergency: Applicable only for health-related or security emergencies.
    High: Issues that could significantly impact passenger safety, comfort, or delay train operations.
    Medium: Problems affecting comfort, convenience, or cleanliness, but not life-threatening or severely disruptive.
    Low: Minor issues or problems that are not urgent and have minimal impact on overall passenger experience.
    Assign the correct department responsible for resolving the issue. Based on the complaint type and subtype, you will choose one of the following departments:

    Traffic
    Civil Engineering
    Mechanical
    Electrical
    Signal and Telecommunications
    Accounts and Finance
    Medical
    Security
    Legal
    Housekeeping
    Catering/Food Department
    If the type or subtype is ambiguous or doesn't directly map to the list, refer to the complaint description for further clarification.

    Priority Rules:

    Emergency: If the complaint involves life-threatening security issues (e.g., "Eve-teasing/Misbehavior with lady passengers," "Dacoity/Robbery/Murder/Riots") or a critical health emergency (e.g., "Medical Assistance"), classify the priority as "Emergency." Assign these cases to the Security or Medical departments, as appropriate.

    High: For complaints involving serious operational issues, such as:

    "Train Delays" impacting large numbers of passengers.
    "Air Conditioner" failure in extreme temperatures.
    Safety-related issues such as "Smoking/Drinking Alcohol/Narcotics," "Coach - Maintenance" (broken windows, jerks, abnormal sounds, etc.), or "Electrical Equipment" failures that pose risks to passengers.
    For "Staff Behavior" involving misconduct or corruption, assign "High" priority and send to Security or Accounts and Finance depending on the context.
    Medium: For complaints impacting comfort and cleanliness, including:

    "Facilities" (e.g., AC malfunction, water unavailability, cleanliness of coach/toilet).
    "Divyangjan Facilities" (e.g., lack of availability of special facilities).
    "Coach - Cleanliness" and "Catering & Vending Service" issues related to food quality, hygiene, or overcharging.
    Assign these complaints to relevant departments, such as Housekeeping, Catering, or Mechanical.
    Low: For minor issues or general feedback:

    "Punctuality" related to minor delays or NTES app feedback.
    "Miscellaneous" complaints with non-urgent concerns.
    General cleanliness, such as "Cockroach/Rodents" or "Coach Interior/Exterior."
    Assign these to Housekeeping or relevant operational departments.
    Detailed Examples:

    Medical Assistance (Emergency)

    Priority: Emergency
    Department: Medical
    Eve-teasing/Misbehavior with lady passengers/Rape

    Priority: Emergency
    Department: Security
    Train Delay

    Priority: Medium (unless the delay severely disrupts the operation)
    Department: Traffic
    AC Malfunction in Coach

    Priority: Medium
    Department: Electrical
    Cockroach/Rodents in Coach

    Priority: Medium
    Department: Housekeeping
    Corruption/Bribery

    Priority: High
    Department: Accounts and Finance
    Food Overcharging

    Priority: Medium
    Department: Catering/Food Department
    Train Windows Broken

    Priority: High
    Department: Mechanical
    In Case of Ambiguity: When the complaint's type or subtype is unclear, read the problem description carefully and infer the most likely priority and department based on the context."""
        

    priority = ""
    department_assigned = ""
    return priority,department_assigned

complaint_classifier_tool = FunctionTool.from_defaults(fn=complaint_classifier)

def output_generator(complaint: dict) -> dict:
    """
    Use this function to classify a complaint, adding priority and assigning a department.
    Takes in a JSON response (as a dictionary) and returns a modified JSON with priority and department.

    Example Input : 
    
    {
        "phone_number": "9123456789",
        "train_number": "12874",
        "pnr_number": "PNR654321",
        "problem_description": "AC in coach not functioning properly.",
        "type": "Facilities",
        "subtype": "AC Malfunction",
        "incident_date": "2023-03-10",
        "incident_time": "16:30"
    }

    Example Output :

    {
        "phone_number": "9123456789",
        "train_number": "12874",
        "pnr_number": "PNR654321",
        "problem_description": "AC in coach not functioning properly.",
        "type": "Facilities",
        "subtype": "AC Malfunction",
        "incident_date": "2023-03-10",
        "incident_time": "16:30",
        "priority": "Medium",
        "department_assigned": "Technical Department"
    }

    """
    mod_complaint = {
        "phone_number": complaint["phone_number"],
        "train_number": complaint["train_number"],
        "pnr_number": complaint["pnr_number"],
        "problem_description": complaint["problem_description"],
        "type": complaint["type"],
        "subtype": complaint["subtype"],
        "incident_date": complaint["incident_date"],
        "incident_time": complaint["incident_time"],
        "priority" : "use complaint_classifier_tool",
        "department" : "use complaint_classifier_tool"
    }

    
    return mod_complaint

output_tool = FunctionTool.from_defaults(fn=output_generator)

In [15]:
from typing import Dict, Optional
from datetime import datetime

def combined_complaint_classifier(complaint: dict) -> dict:
    """
    You are an intelligent complaint classification system designed to work for Rail Madad, the IRCTC's complaint grievance platform. Your primary task is to:

    1. Determine the priority of a complaint based on the details provided. The priorities are categorized as follows:
        - Emergency: Applicable for health-related or security emergencies.
        - High: Issues that could significantly impact passenger safety, comfort, or delay train operations.
        - Medium: Problems affecting comfort, convenience, or cleanliness, but not life-threatening or severely disruptive.
        - Low: Minor issues or problems that are not urgent and have minimal impact on overall passenger experience.

    2. Assign the correct department responsible for resolving the issue. The departments available are:
        - Traffic
        - Civil Engineering
        - Mechanical
        - Electrical
        - Signal and Telecommunications
        - Accounts and Finance
        - Medical
        - Security
        - Legal
        - Housekeeping
        - Catering/Food Department

    The decision for assigning priority and department depends on the type and subtype of the complaint. If the type or subtype is ambiguous, the complaint description should be analyzed for further clarification.

    Priority Rules:

    - Emergency: For life-threatening security issues (e.g., "Eve-teasing," "Dacoity," "Robbery," etc.) or critical health emergencies (e.g., "Medical Assistance"). These cases are assigned to the Security or Medical departments, depending on the issue.
    - High: For serious operational or safety issues (e.g., "Train Delays," "Air Conditioner Failure in Extreme Conditions," or safety risks related to electrical equipment). Issues of misconduct or corruption (e.g., "Corruption/Bribery") also fall under this category. These may be assigned to departments like Traffic, Electrical, or Accounts and Finance.
    - Medium: For issues impacting comfort or cleanliness (e.g., "AC Malfunction," "Coach Cleanliness," "Food Quality," or "Facilities"). These are assigned to relevant departments such as Electrical, Housekeeping, or Catering.
    - Low: For minor issues or feedback that don't require urgent attention (e.g., minor delays, cleanliness complaints about cockroaches, rodents, etc.). These are assigned to departments like Housekeeping or relevant operational ones.

    Example Input: 
    
    {
        "phone_number": "9123456789",
        "train_number": "12874",
        "pnr_number": "PNR654321",
        "problem_description": "AC in coach not functioning properly.",
        "type": "Facilities",
        "subtype": "AC Malfunction",
        "incident_date": "2023-03-10",
        "incident_time": "16:30"
    }

    Example Output:

    {
        "phone_number": "9123456789",
        "train_number": "12874",
        "pnr_number": "PNR654321",
        "problem_description": "AC in coach not functioning properly.",
        "type": "Facilities",
        "subtype": "AC Malfunction",
        "incident_date": "2023-03-10",
        "incident_time": "16:30",
        "priority": "Medium",
        "department_assigned": "Electrical"
    }

    This function returns a modified JSON containing the original complaint information with added "priority" and "department_assigned" fields based on the classification logic.
    """
    priority = ""
    department_assigned = ""

    # Prepare modified complaint with original details and classified fields.
    mod_complaint = {
        "phone_number": complaint["phone_number"],
        "train_number": complaint["train_number"],
        "pnr_number": complaint["pnr_number"],
        "problem_description": complaint["problem_description"],
        "type": complaint["type"],
        "subtype": complaint["subtype"],
        "incident_date": complaint["incident_date"],
        "incident_time": complaint["incident_time"],
        "priority": priority,
        "department_assigned": department_assigned
    }

    return mod_complaint


complaint_classifier_tool = FunctionTool.from_defaults(fn=combined_complaint_classifier)

In [16]:
from llama_index.core.agent import ReActAgent, FunctionCallingAgentWorker

react_agent = ReActAgent.from_tools(tools=[complaint_classifier_tool],llm=llm, verbose=True)

In [17]:
while True:
    user_input = input("You : ")
    if user_input.lower() == "quit":
        break

    response = react_agent.chat("Always give only json response, anything else in the output will result in punishment "+ user_input)
    print("Bot : " , response)

> Running step 4209802a-a1c6-4c5e-afac-fd1d4a7b66bd. Step input: Always give only json response, anything else in the output will result in punishment {         "phone_number": "9023456789",         "train_number": "12005",         "pnr_number": "PNR123789",         "problem_description": "Lights in the coach are not working.",         "type": "Electrical Equipment",         "subtype": "Lights",         "incident_date": "2023-09-01",         "incident_time": "19:20" }
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: combined_complaint_classifier
Action Input: {'complaint': AttributedDict([('phone_number', '9023456789'), ('train_number', '12005'), ('pnr_number', 'PNR123789'), ('problem_description', 'Lights in the coach are not working.'), ('type', 'Electrical Equipment'), ('subtype', 'Lights'), ('incident_date', '2023-09-01'), ('incident_time', '19:20')])}
Observation: {'phone_number': '9023456789', 'train_number': '120

In [ ]:
#sentiment analysis,
# storing compaints in a database and retrieving it back,
#customer general chatbot, acknowledgement of complaint and preliminary messages 
points for departments working well based on blah blah
future scope = { resource allocation based on user issue........what are required both human and material Resource
probabilistic image caption + text description - pass it to llm for better decision making }





In [ ]:
{
 "phone_number": "7223456789", 
 "train_number": "12958", 
 "pnr_number": "PNR789654", 
 "problem_description": "Food quality was very poor.", 
 "type": "Catering & Vending Service", 
 "subtype": "Food Quality & Quantity", 
 "incident_date": "2023-09-01", 
 "incident_time": "13:15", 
 "priority": "Medium", 
 "department_assigned": "Catering/Food Department"
 }

In [ ]:
Type                                                            Subtype

Medical Assistance                                              Medical Assistance

Security                                                        Eve-teasing/Misbehaviour
                                                                Eve-teasing/Misbehaviour with lady passengers/Rape
                                                                Theft of Passengers Belongings/Snatching
                                                                Unauthorized person in Ladies/Disabled Coach/SLR/Reserve Coach Harassment/Extortion by Security Personnel/Railway personnel
                                                                Nuisance by Hawkers/Beggar/Eunuch
                                                                Luggage Left Behind/Unclaimed/Suspected Articles
                                                                Passenger Missing/Not responding call
                                                                Smoking/Drinking Alcohol/Narcotics
                                                                Dacoity/Robbery/Murder/Riots
                                                                Quarrelling/Hooliganism Passenger fallen down
                                                                Nuisance by passenger
                                                                Misbehaviour
                                                                Others   

Divyangjan Facilities                                           Divyangjan coach unavaila
                                                                Divyangjan coach unavailability 
                                                                Divyangjan toilet/washbasin 
                                                                Braille signage in coach
                                                                Others

Facilities for Women with Special needs                         Baby Food 

Electrical Equipment                                            Lights
                                                                Air Conditioner
                                                                Fans
                                                                Charging Points
                                                                Others

Coach - Cleanliness                                               Toilet
                                                                Washbasin
                                                                Cockroach / Rodents
                                                                Coach Interior
                                                                Coach Exterior
                                                                Others

Punctuality                                                     NTES APP
                                                                Late Running
                                                                Others



Water Availability                                              Packaged Drinking Water / Rail Neer
                                                                Toilet
                                                                Washbasin
                                                                Others

Staff Behaviour                                                 Staff Behaviour

Corruption / Bribery                                            Corruption / Bribery 

Coach - Maintenance                                             Window/Seat Broken 
                                                                Window/Door locking problem 
                                                                Tap leaking/Tap not working 
                                                                Broken/Missing Toilet Fittings 
                                                                Jerks/Abnormal Sound
                                                                Others


Catering & Vending Service                                      Overcharging
                                                                Service Quality & Hygiene 
                                                                Food Quality & Quantity
                                                                E-Catering
                                                                Food & Water Not Available
                                                                Others


Bed Roll                                                        Dirty / Torn 
                                                                Overcharging 
                                                                Non Availability
                                                                Others

Miscellaneous                                                   Miscellaneous